In [ ]:
import os
os.listdir('data')

# Wage data introduced
All data is in the data directory. This data was downloaded from the ISLR and MASS R packages

In [ ]:
# Bring data into workspace and replicate plots
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df_wage = pd.read_csv('data/wage.csv')

In [ ]:
df_wage.head(10)

In [ ]:
df_wage.info()

In [ ]:
df_wage.describe(include=['object'])

In [ ]:
# Look at all available plotting styles
plt.style.available

In [ ]:
plt.style.use("ggplot")

In [ ]:
# need to reshape data to plot correctly
df_edu = df_wage.pivot(columns='education', values='wage')

In [ ]:
df_edu.head(15)

In [ ]:
# importing statsmodels library to fit lowess curve through data
import statsmodels.api as sm

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(14,6))
df_wage.plot.scatter('age', 'wage', ax=ax[0])
lowess = sm.nonparametric.lowess(df_wage['wage'], df_wage['age'], frac=.2)
ax[0].plot(lowess[:, 0], lowess[:, 1])

df_wage.plot.scatter('year', 'wage', ax=ax[1])
year_median = df_wage.groupby('year')['wage'].median()
ax[1].plot(year_median)

boxplot = df_edu.plot.box(ax=ax[2], rot=45, patch_artist=True)
colors = ['lightblue', 'green', 'yellow', 'blue', 'red']
for artist, color in zip(boxplot.artists, colors):
    artist.set_facecolor(color)

In [ ]:
import seaborn as sns

In [ ]:
# Similar plots to those above
sns.lmplot(x='age', y='wage', data=df_wage, hue='education')
sns.lmplot(x='year', y='wage', data=df_wage, ci=99.99, hue='education');

In [ ]:
sns.boxplot(x='education', y='wage', data=df_wage);

In [ ]:
df1 = df_wage[['age', 'year', 'education', 'wage']]
df1.head()

In [ ]:
df_melt = pd.melt(df1, id_vars=['education', 'wage'])

In [ ]:
df_melt.head()

In [ ]:
seaborn_grid = sns.lmplot(x='value', y='wage', col='variable', hue='education', data=df_melt, sharex=False)
seaborn_grid.fig.set_figwidth(8)

left, bottom, width, height = seaborn_grid.fig.axes[0]._position.bounds
left2, bottom2, width2, height2 = seaborn_grid.fig.axes[1]._position.bounds
left_diff = left2 - left
seaborn_grid.fig.add_axes((left2 + left_diff, bottom, width, height))

sns.boxplot(x='education', y='wage', data=df_wage, ax = seaborn_grid.fig.axes[2])
ax2 = seaborn_grid.fig.axes[2]
ax2.set_yticklabels([])
ax2.set_xticklabels(ax2.get_xmajorticklabels(), rotation=30)
ax2.set_ylabel('')
ax2.set_xlabel('');

leg = seaborn_grid.fig.legends[0]
leg.set_bbox_to_anchor([0, .1, 1.5,1])

# Regression vs Classification


In [ ]:
df_smarket = pd.read_csv('data/smarket.csv')

In [ ]:
df_smarket.head()

In [ ]:
# Put all lags in one column. Make 'Tidy' Data
df_smarket_pivot = pd.melt(df_smarket, 
                           id_vars='Direction', 
                           value_vars=['Lag1', 'Lag2', 'Lag3'], 
                           var_name='Lag Type', 
                           value_name='Pct Change')

In [ ]:
df_smarket_pivot.head()

In [ ]:
g = sns.FacetGrid(df_smarket_pivot, col="Lag Type", aspect=.6)
g = g.map(sns.boxplot, "Direction", "Pct Change")

In [ ]:
sns.catplot(x="Lag Type", y="Pct Change", hue="Direction",data=df_smarket_pivot, kind="box", aspect=1.4)

# Clustering
No longer interested in prediction - looking to discover underlying similarities in the data

In [ ]:
df_genes = pd.read_csv('data/nci60_data.csv', index_col=0)

In [ ]:
df_genes.head()

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(2)

In [ ]:
df_genes_transformed = pd.DataFrame(pca.fit_transform(df_genes), columns=['PC1', 'PC2'])

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(4)

In [ ]:
kmeans.fit(df_genes_transformed)

In [ ]:
kmeans.labels_

In [ ]:
df_genes_transformed['cluster'] = kmeans.labels_

In [ ]:
sns.lmplot(x='PC1', y='PC2', data=df_genes_transformed, fit_reg=False, hue='cluster', 
           scatter_kws={"marker": "D", "s": 100})

## Advertising Data
The advertising data consists of product sales from 200 markets and their associated tv, radio, and newspaper advertising budgets. What kind of relationship can be seen between advertising budget and sales

In [ ]:
df_adv = pd.read_csv('data/Advertising.csv')
df_adv.head(10)

In [ ]:
df_adv_new = pd.melt(df_adv, value_vars=['TV', 'Radio', 'Newspaper'], id_vars='Sales', value_name='adv_budget')
df_adv_new.head(10)

In [ ]:
sns.lmplot(x='adv_budget', y='Sales', data=df_adv_new, hue='variable', fit_reg=False);

In [ ]:
lm = sns.lmplot(x='adv_budget', y='Sales', data=df_adv_new, col='variable', sharey=False, sharex=False, lowess=True);
axes = lm.axes
for i, ax in enumerate(axes[0]):
    ax.set_xlim(0,)
    ax.set_title(lm.col_names[i])
    ax.set_xlabel('Advertising Budget')

# Training Data vs Testing Data
**Training Data** - Data used to build a prediction model. Should not be used to validate the model.  
**Testing Data** - Data used to determine the usefulness of the model. Validates the model. This data is unseen during model building phase.



In [ ]:
num_points = 30

In [ ]:
np.random.seed(12345)
x = np.linspace(1,13, num_points).reshape(-1, 1)
error = np.random.randn(num_points, 1) * num_points
f = lambda x: (x - 2) * (x - 6) * (x - 12)

y = f(x) + error

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly = PolynomialFeatures(degree=10)
X = poly.fit_transform(x)

In [ ]:
obs_nums = np.arange(0, num_points)
np.random.shuffle(obs_nums)

top_70 = int(num_points * .7)
rand_train = np.sort(obs_nums[:top_70])
rand_test = np.sort(obs_nums[top_70:])

In [ ]:
X_train = X[rand_train]
X_test = X[rand_test]
y_train = y[rand_train]
y_test = y[rand_test]

In [ ]:
linreg = LinearRegression()
linreg.fit(X_train[:, :2], y_train)
y_train_2 = linreg.predict(X_train[:, :2])
y_test_2 = linreg.predict(X_test[:, :2])

linreg.fit(X_train[:, :4], y_train)
y_train_4 = linreg.predict(X_train[:, :4])
y_test_4 = linreg.predict(X_test[:, :4])

linreg.fit(X_train, y_train)
y_train_10 = linreg.predict(X_train)
y_test_10 = linreg.predict(X_test)

In [ ]:
errors_train= np.array([np.mean((y_train - y_train_2) ** 2),
                        np.mean((y_train - y_train_4) ** 2),
                        np.mean((y_train - y_train_10) ** 2)])
errors_train = np.column_stack(([2, 4, 10], errors_train))

errors_test = np.array([np.mean((y_test - y_test_2) ** 2),
                        np.mean((y_test - y_test_4) ** 2),
                        np.mean((y_test - y_test_10) ** 2)])
errors_test = np.column_stack(([2, 4, 10], errors_test))

In [ ]:
plt.scatter(X_train[:,1], y_train, c='b', label='Train')
plt.scatter(X_test[:,1], y_test, c='r', label = 'Test')
plt.plot(X_train[:,1], y_train_2, label = '1')
plt.plot(X_train[:,1], y_train_4, label = '3')
plt.plot(X_train[:,1], y_train_10, label = '10')
plt.legend(loc=2);

In [ ]:
plt.plot(errors_train[:, 0], errors_train[:, 1], label = 'Train')
plt.plot(errors_test[:, 0], errors_test[:, 1], label = 'Test')
plt.hlines(900, 2, 10, label = 'Best', linestyle = '--')

plt.legend()
plt.title("Training and Test MSE")
plt.xlabel('Flexibility')
plt.ylabel('MSE');

## Problem (advanced)
<span style="color:green">Write a function that a takes a list/array of how many parameters to fit a linear regression model for the above data and outputs the two plots above.</span>

# Exercises

# Problem 3
3a. Hand-picked points to show the 5 curves

In [ ]:
bias = np.array([6, 4, 2, 1, .5, .1])
variance = bias[::-1]
training_error = bias * 1.1
test_error = np.array([7, 5, 2, 2, 5, 7])
irreducible_error = np.ones(6) * 1.5

In [ ]:
df_3 = pd.DataFrame({'bias': bias,
              'variance':variance,
             'training_error': training_error,
             'test_error':test_error,
             'irreducible_error': irreducible_error})

In [ ]:
df_3.plot()
plt.xlabel('Complexity')

# Problem 7

In [ ]:
df_7 = pd.DataFrame({'x1': [0, 2, 0, 0, -1, 1], 'x2':[3, 0, 1, 1, 0, 1], 'x3':[0, 0, 3, 2, 1, 1], 
                     'y':['R', 'R', 'R', 'G', 'G', 'R']})
df_7

In [ ]:
# PART a
# Get x1, x2, x3 from the above dataframe. Subtract (0, 0, 0) from it and square each dimension
dist = (df_7.values[:, :3] - np.array([0, 0, 0])) ** 2

# Sum across the rows and make sure the type is float
summed_distance = dist.sum(axis=1).astype('float')

# Take square root to get euclidean distance
euclidean_dist = np.sqrt(summed_distance)
euclidean_dist

# Problem 8

## Part a and b

In [ ]:
df_college = pd.read_csv('data/college.csv', index_col=0)
df_college.head()

## Part c

In [ ]:
# i
df_college.describe()

In [ ]:
# ii
sns.pairplot(df_college.iloc[:, :10]);

In [ ]:
# iii
sns.boxplot('Private', 'Outstate', data=df_college);

In [ ]:
df_college

In [ ]:
# iv
# Next line produces No/Yes categories based on a boolean(0/1) and saves it as a DataFrame column
df_college['Elite'] = pd.Categorical(np.where(df_college['Top10perc'] > 50, 'Yes', 'No'))
print(df_college['Elite'].value_counts())
sns.boxplot('Elite', 'Outstate', data=df_college);

In [ ]:
# v
fig , ax = plt.subplots(2, 2, figsize=(12,8))
ax[0, 0].hist(df_college['Accept'] / df_college['Apps'] , bins=5)
ax[0, 0].set_title('Percentage Accepted')

ax[0, 1].hist(df_college['Accept'] / df_college['Apps'] , bins=10)
ax[0, 1].set_title('Percentage Accepted')

ax[1, 0].hist(df_college['Accept'] / df_college['Apps'] , bins=15)
ax[1, 0].set_title('Percentage Accepted')

ax[1, 1].hist(df_college['Accept'] / df_college['Apps'] , bins=20)
ax[1, 1].set_title('Percentage Accepted');

In [ ]:
# vi
# Acceptance rate and Graduation rate are negatively correlated
df_college['Accept_Rate'] = df_college['Accept'] / df_college['Apps']
sns.lmplot('Accept_Rate', 'Grad.Rate', data=df_college);

# Problem 9

In [ ]:
df_auto = pd.read_csv('data/auto.csv')

In [ ]:
df_auto.info()

a) Quantitative vs Qualitative Predictors  
Quantitative - mpg, cylinders, displacement, horsepower, weight, acceleration  
Qualitative - year, origin, name

In [ ]:
# b and c - get the range and std of each quantitative predictor
df_auto.describe()

In [ ]:
# d
pd.concat((df_auto.iloc[:10], df_auto.iloc[85:])).describe()

In [ ]:
# e
# Horsepower and displacement have a very strong postive linear relationship. Horsepower and mpg 
# have a strong negative relationship
sns.pairplot(df_auto)

f) Looking at the pair plots above, mpg seems to have some relationship with just about all the other predictors.

# Problem 10

In [ ]:
df_boston = pd.read_csv('data/boston.csv')

In [ ]:
df_boston.shape

This data frame contains the following columns:

crim
per capita crime rate by town.

zn
proportion of residential land zoned for lots over 25,000 sq.ft.

indus
proportion of non-retail business acres per town.

chas
Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).

nox
nitrogen oxides concentration (parts per 10 million).

rm
average number of rooms per dwelling.

age
proportion of owner-occupied units built prior to 1940.

dis
weighted mean of distances to five Boston employment centres.

rad
index of accessibility to radial highways.

tax
full-value property-tax rate per \$10,000.

ptratio
pupil-teacher ratio by town.

black
1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town.

lstat
lower status of the population (percent).

medv
median value of owner-occupied homes in \$1000s.



In [ ]:
# The pair plot is too large. Lets plot correlations and 
df_boston.corr()

c) There are no very strong relationships (> .9) with crime. The highest two are **rad**, **tax** and **lstat**. Crime is correlated with density of population (from what I've read before) so rad might be representative of how dense the population is if you are close to highways. Tax rates are generally higher the closer you are to a city center so higher tax rates might imply denser populations. And lower status (lstat) makes sense since more crime is committed by those less well off.

d) Below are the towns that have a max for each of the predictors. Seems there are some limitation in the data such that 132 towns have exactly 24 as a value for rad and 121 towns have exactly 396.9 as a value for black. Crime also seems to be strangely distributed with nearly all values hovering around 0 and a few serveral orders of magnitude greater.

In [ ]:
df_boston.loc[df_boston.idxmax().unique()].style.highlight_max()

In [ ]:
(df_boston['rad'] == 24).sum(), (df_boston['black'] == 396.9).sum()

In [ ]:
plt.hist(df_boston['crim']);

In [ ]:
# e
df_boston['chas'].sum()

In [ ]:
# f
df_boston['ptratio'].median()

In [ ]:
# g
# rad and black are both those suspicious maximum values and crim is a ridiculous outlier. More evidence
# of bad data
df_boston.loc[df_boston['medv'].idxmin()]

In [ ]:
# h
(df_boston['rm'] > 7).sum(), (df_boston['rm'] > 8).sum()

In [ ]:
pd.DataFrame({'More than 8 rooms': df_boston[df_boston['rm'] > 8].describe().loc['50%'], 
              '8 or less rooms' : df_boston[df_boston['rm'] <= 8].describe().loc['50%']})

Most of the predictors are the similar in both groups except crime and median value of homes - both about double for more than 8 rooms.